For subzone: calculate population density
For pri sch, sec sch, shopping malls: calculate number of pri sch, sec schs, shopping malls in the subzone
For mrt-station: calculate distance to the nearest mrt station for each property

In [8]:
import numpy as np
import pandas as pd
import geopy.distance

In [9]:
df_pri_sch = pd.read_csv('../data/auxiliary-data/sg-primary-schools.csv')
df_pri_sch = df_pri_sch.drop(["name", "lat", "lng", 'planning_area'], axis=1)
print(df_pri_sch.head())

           subzone
0   woodlands east
1      yishun west
2        cheng san
3  alexandra north
4       anchorvale


In [10]:
pri_sch_cleaned = df_pri_sch.value_counts().to_frame(name="pri_sch")
print(pri_sch_cleaned)

                     pri_sch
subzone                     
woodlands east             6
tampines east              6
yunnan                     5
punggol town centre        4
bedok north                4
...                      ...
keat hong                  1
katong                     1
kampong ubi                1
kampong java               1
midview                    1

[109 rows x 1 columns]


In [11]:
df_sec_sch = pd.read_csv('../data/auxiliary-data/sg-secondary-schools.csv')
df_sec_sch = df_sec_sch.drop(["name", "lat", "lng", 'planning_area'], axis=1)
print(df_sec_sch.head())

          subzone
0  woodlands east
1     yishun west
2       cheng san
3     bedok north
4         malcolm


In [12]:
sec_sch_cleaned = df_sec_sch.value_counts().to_frame(name="sec_sch")

In [13]:
print(sec_sch_cleaned)

                sec_sch
subzone                
tampines east         5
marymount             3
bedok south           3
tampines west         3
woodlands east        3
...                 ...
joo seng              1
jelebu                1
hougang east          1
henderson hill        1
yunnan                1

[101 rows x 1 columns]


In [14]:
df_subzone = pd.read_csv('../data/auxiliary-data/sg-subzones.csv')
df_subzone["population_density"] = df_subzone['population']/df_subzone["area_size"]
df_subzone = df_subzone.drop(['area_size', 'population', 'planning_area'],axis=1).set_index("name")
print(df_subzone.head())

                        population_density
name                                      
ang mo kio town centre        15178.289681
cheng san                     29371.141572
chong boon                    24568.885592
kebun bahru                   21616.972477
sembawang hills                7657.909447


In [15]:
df_mrt_station = pd.read_csv('../data/auxiliary-data/sg-mrt-stations.csv')
df_mrt_station = df_mrt_station.drop(["name", "lat", "lng", 'planning_area', 'code', 'line', 'opening_year'], axis=1)
print(df_mrt_station.head())

         subzone
0    dhoby ghaut
1     bras basah
2      city hall
3  marina centre
4         nicoll


In [16]:
mrt_station_cleaned = df_mrt_station.value_counts().to_frame(name="mrt_station")
print(mrt_station_cleaned.head())

                   mrt_station
subzone                       
maritime square              4
aljunied                     3
serangoon central            3
geylang east                 3
city hall                    3


In [17]:
df_train = pd.read_csv('../data/train_cleaned.csv')
df_test = pd.read_csv('../data/test_cleaned.csv')

In [19]:
df_train.merge(pri_sch_cleaned, how='left',left_on="subzone",right_on="subzone")\
    .merge(sec_sch_cleaned, how='left',left_on="subzone",right_on="subzone")\
    .merge(mrt_station_cleaned, how='left',left_on="subzone",right_on="subzone")\
    .merge(df_subzone, how='left',left_on="subzone",right_on="name")\
    .fillna({'pri_sch':0, 'sec_sch':0, 'mrt_station':0})

,Unnamed: 0,Unnamed: 0.1,title,address,property_name,property_type,num_beds,num_baths,size_sqft,furnishing,...,subzone,planning_area,price,built_year,tenure,available_unit_types,pri_sch,sec_sch,mrt_station,population_density
0,0,0,hdb flat for sale in 866 yishun street 81,sembawang / yishun (d27),866 yishun street 81,hdb,3.0,2.0,1115,unspecified,...,yishun south,yishun,514500.0,1988.0,99-year leasehold,NaN,2.0,2.0,0.0,31517.683928
1,1,1,hdb flat for sale in 506b serangoon north aven...,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,4.0,2.0,1575,unspecified,...,serangoon north,serangoon,995400.0,1992.0,99-year leasehold,"1, 2, 3, 4, 5, 6 br",1.0,0.0,0.0,23280.268731
2,2,2,4 bed condo for sale in meyerhouse,128 meyer road,meyerhouse,condo,4.0,6.0,3070,partial,...,mountbatten,marine parade,8485000.0,2022.0,freehold,"studio, 3, 4, 5, 6 br",0.0,1.0,0.0,5829.779777
3,3,3,3 bed condo for sale in leedon green,26 leedon heights,leedon green,condo,3.0,2.0,958,partial,...,farrer court,bukit timah,2626000.0,2023.0,freehold,"studio, 1, 2, 3, 4 br",1.0,1.0,1.0,11059.413028
4,4,4,2 bed condo for sale in one bernam,1 bernam street,one bernam,condo,2.0,1.0,732,unspecified,...,anson,downtown core,1764000.0,2026.0,99-year leasehold,"studio, 1, 2, 3, 4, 5 br",0.0,0.0,0.0,775.193798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19632,19641,19641,2 bed condo for sale in lentor modern,lentor central,lentor modern,condo,2.0,2.0,635,unspecified,...,tagore,ang mo kio,1050000.0,2026.0,99-year leasehold,"1, 2, 3, 4 br",0.0,0.0,1.0,2384.380061
19633,19642,19642,2 bed condo for sale in mori,223 guillemard road,mori,condo,2.0,2.0,883,unspecified,...,lavender,kallang,2087400.0,2026.0,freehold,"1, 2, 3, 4 br",0.0,0.0,2.0,12802.219580
19634,19643,19643,4 bed condo for sale in pullman residences newton,18 dunearn road,pullman residences newton,condo,4.0,4.0,1378,unspecified,...,moulmein,novena,4193700.0,2023.0,freehold,"studio, 1, 2, 3, 4, 5 br",1.0,0.0,1.0,6901.669759
19635,19644,19644,hdb flat for sale in 691d woodlands drive 73,admiralty / woodlands (d25),admiralty flora,hdb,3.0,2.0,1205,unfurnished,...,woodlands east,woodlands,754800.0,2017.0,99-year leasehold,"2, 3, 4 br",6.0,3.0,1.0,38762.482867
